<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Practica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 4**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

**Requisitos mínimos:**

* **Funciones** que generen trayectorias tipo Brownian Motion (**BM**), Correlated Random Walk (CRW) y Lévy Flight (LF).
* Cada una de las funciones deberá tomar como parámetros el **numero de pasos**, la **velocidad** y **posición inicial**.
  * Además la funciones para **CRW** y **LF** deberán tomar como parámetro el **coeficiente** para la distribución Cauchy.
  * Por último, la función para **LF** también deberá también aceptar como parámetro el **exponente Lévy** (alpha).
* **Funciones** para calcular las métricas **Path length** and **Mean Squared Displacement**.

**El dashboard degerá contener al menos los siguientes elementos:**

* Un panel para desplegar la trayectoria en 3D
* Un panel para desplegar la métrica de elección de la trayectoria.
* Drop down menu (u otro tipo de selector) que permita elegir la métrica a calcular de la trayectoria bajo análisis (path length o mean squared displacement).
* Radio Buttons (u otro tipo de selector) que permita elegir el tipo de trayectoria a generar y análizar.
* Widgets que permitan introducir los valores enteros o de punto flotante (segun sea el caso) para ajustar los parámetros de las trayectorias. Estos widgets deberán mostrarse de manera dinámica, es decir, solo deberan mostrarse los parametros relevantes para cada tipo de trayectoria.

## **Módulos**

In [1]:
## Modulos##

import math
import numpy as np
import pandas as pd
import matplotlib. pyplot as plt
import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## **Classes**

In [2]:
## CLASES ##

class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## **Functions**

In [3]:
## funcion ##

def bm_2d(pasos=1000, velocity=6,posIniX=0,posIniY=0):
  #se utiliza clase
  velocity = Vec2d(velocity,0)

  # estructura de dataframe pd #
  BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos','z_pos','msd','pl'])
  temp_df = pd.DataFrame([{'x_pos':posIniX,'y_pos':posIniY,'z_pos':0,'msd':0,'pl':0}])
  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  #rumbos aleatorios
  for i in range(0,pasos-1):

    #from -pi a pi que posibilidades d giro presentan los 360 grados de posibilidades
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity=velocity.rotated(turn_angle)
    temp_df= pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x,'y_pos':BM_2d_df.y_pos[i]+velocity.y,'z_pos':i+1,'msd':0,'pl':0}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  return BM_2d_df

In [4]:
## Caminata aleatoria c ##

def CRW_2d(pasos=1000, velocity=6, posIniX=0, posIniY=0, factorCauchy=0.4,):
  #se usa clase
  velocity = Vec2d(velocity,0)

  #estructura de dataframe pd
  CRW_2d_df = pd.DataFrame(columns=['x_pos','y_pos', 'z_pos', 'msd', 'pl'])
  temp_df = pd.DataFrame([{'x_pos':posIniX, 'y_pos':posIniY, 'z_pos':0,'msd':0, 'pl':0}])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)
  ListaPDTrayectorias = []
  distribucionDireccion = wrapcauchy(factorCauchy,0,scale=1)
  listaDirecciones = distribucionDireccion.rvs(size=pasos)
  for i in range(0, pasos -1):
    turn_angle = listaDirecciones[i]
    velocity=velocity.rotated(turn_angle)
    temp_df= pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i]+velocity.x,'y_pos':CRW_2d_df.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)
  return CRW_2d_df

In [5]:
## Levy Flight ##

def LevyFlyPd(pasos,speed, miu, levyExponent, levyBeta, factorCauchy, posIniX=0, posIniY=0):
  s_pos = [0,0]
  velocity = Vec2d(speed,0)

  # Instancia de objeto Wrapcauchy
  distribucionDireccion = wrapcauchy(0.4,0,scale=1)

  # Cálculo de giros para trayectorias
  listaDirecciones = distribucionDireccion.rvs(size=pasos)

  # Instancia de objeto Levy_stable
  distribucionPasos = levy_stable(alpha=levyExponent, beta=levyBeta, loc=miu)

  # Cálculo de pasos para la trayectoria
  listaCantidadPasos = distribucionPasos.rvs(size=pasos)

  # Creamos el DataFrame con Pandas
  pdLevy = pd.DataFrame(columns=['x_pos','y_pos', 'z_pos', 'msd', 'pl'])
  temp3d = pd.DataFrame([{'x_pos':posIniX,'y_pos':posIniY,'z_pos':0,'msd':0,'pl':0}])
  pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
  PasosPendientes=0
  IndicePasos = 0
  IndiceCambiosDireccion = 0

  for i in range(0,pasos-1):
    if PasosPendientes < 1:
      velocity=velocity.rotated(listaDirecciones[IndiceCambiosDireccion])
      IndiceCambiosDireccion=IndiceCambiosDireccion + 1
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
      pdLevy = pd.concat([pdLevy, temp3d], ignore_index=True)
      PasosPendientes=int(listaCantidadPasos[IndicePasos])
      IndicePasos=IndicePasos+1
    else:
      temp3d = pd.DataFrame([{'x_pos':pdLevy.x_pos[i]+velocity.x,'y_pos':pdLevy.y_pos[i]+velocity.y,'z_pos':i,'msd':0,'pl':0}])
      pdLevy = pd.concat([pdLevy,temp3d], ignore_index=True)
      PasosPendientes=PasosPendientes-1

    return pdLevy

In [6]:
## cálculo de distancia ##

def distanciaXYmean(x1=0,y1=0,x2=1,y2=1):
  distancia = math.sqrt((x2-x1)**2+(y2-y1)**2)

  return distancia

def distanciaXY(x1=0,y1=0,x2=1,y2=1):
  distancia = math.sqrt((x2-x1)**2+(y2-y1)**2)
  return distancia